In [1]:
# basic packages
import pandas as pd
import numpy as np

# bigquery connection
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
file = './My First Project-5abe6994fdf1.json'
credentials = service_account.Credentials.from_service_account_file(file)
project_id = 'keen-bucksaw-273300'
client = bigquery.Client(credentials= credentials,project=project_id)

# Plotly mapbox public token
mapbox_access_token = "pk.eyJ1IjoicGxvdGx5bWFwYm94IiwiYSI6ImNqdnBvNDMyaTAxYzkzeW5ubWdpZ2VjbmMifQ.TXcBE-xg9BFdV2ocecc_7g"

In [6]:
# querying downsampled data from bigquery database

years = [2013, 2014, 2015, 2016, 2017, 2018]

for year in years:
    start = "'" + str(year) + "-01-01'"
    end = "'" + str(year) + "-12-31'"
    
    query = f"""

    SELECT *
    FROM `bigquery-public-data.new_york_citibike.citibike_trips`
    WHERE rand() < 1/10
    AND starttime between {start} and {end}

    """

    query_job = client.query(query)
    results = query_job.result() 
    df = results.to_dataframe()
    df.to_csv(f'./data/initial/{str(year)}.csv', index = False)

In [ ]:
# bigquery database has gap between 2016-10 and 2017-03, and stops at 2018-05
# downloading remaining data from citibike aws

df_all = pd.DataFrame()
errors = []

from bs4 import BeautifulSoup, SoupStrainer
import requests

url = "https://s3.amazonaws.com/tripdata/"

page = requests.get(url)    
data = page.text
soup = BeautifulSoup(data, 'html.parser')

links = []
for link in soup.find_all('key'):
    links.append(link.get_text())
    
# need to find less hideous way of selecting links:

gap = links[41:47]
for link in links[61:82]:
    gap.append(link)

for link in gap:
    print(f'fetching {link}')
    URL = f'https://s3.amazonaws.com/tripdata/{link}'
    
    url = urlopen(URL)
    output = open('zipFile.zip', 'wb')    # note the flag:  "wb"        
    output.write(url.read())
    output.close()

    zip_file = ZipFile('zipFile.zip')
    dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
           for text_file in zip_file.infolist()
           if text_file.filename.endswith('.csv')}
    
    try:
        if 'csv' in link:
            file = link.replace('.zip', '')
        else:            
            file = link.replace('zip', 'csv')
        
        print(f'downloading {file}')    
        dfs[file].to_csv(f'./data/citi/{file}', index = False)
    except:
        errors.append(link)

In [ ]:
with open(f'./data/citi/201809-citibike-tripdata.csv', "r") as f:
    reader = csv.reader(f)
    i = next(reader)
    
columns = [i.replace(' ', '_') for i in i[1:]]
columns

In [ ]:
# importing and combining citibike AWS data

df_citi = pd.DataFrame()
filelist = os.listdir('./data/citi')

for file in filelist:
    if file.endswith('.csv'):
        
        print(f'reading {file}')
        
        temp_df = pd.read_csv(f'./data/citi/{file}').drop(columns = {'Unnamed: 0'})
                
        temp_df.columns = columns
        
        df_citi = df_citi.append(temp_df.sample(frac = 0.1))

df_citi.to_csv('./data/citi/combined.csv', index = False)

In [3]:
# query for people going on bike rides

query = """
with a as (

    select start_station_id
    , end_station_id
    , start_station_latitude
    , start_station_longitude
    , end_station_latitude
    , end_station_longitude
    , tripduration
    , bikeid
    , starttime
    , stoptime
    , birth_year
    , gender
    FROM `bigquery-public-data.new_york_citibike.citibike_trips`
    where start_station_id is not null
    and end_station_id is not null
    and birth_year is not null  
    
), b as (

    select start_station_id
    , end_station_id
    , tripduration
    , bikeid
    , starttime
    , stoptime
    , birth_year
    , gender
    FROM `bigquery-public-data.new_york_citibike.citibike_trips`
    where start_station_id is not null
    and end_station_id is not null
    and birth_year is not null
)

select a.start_station_id
, a.end_station_id
, a.start_station_latitude
, a.start_station_longitude
, a.end_station_latitude
, a.end_station_longitude
, a.starttime as a_start
, b.starttime as b_start
, a.stoptime as a_stop
, b.stoptime as b_stop
, a.tripduration as a_duration
, b.tripduration as b_duration
, a.birth_year as a_yob
, a.gender as a_gender
, b.birth_year as b_yob
, b.gender as b_gender
from a join b on a.start_station_id = b.start_station_id
		and a.end_station_id = b.end_station_id
		and a.bikeid <> b.bikeid
        and a.starttime > b.starttime
		and timestamp_diff(timestamp(a.starttime), timestamp(b.starttime), second) <= 30 
		and abs(timestamp_diff(timestamp(a.stoptime), timestamp(b.stoptime), second)) <= 30  
"""

query_job = client.query(query)
results = query_job.result() 
df = results.to_dataframe()
df.to_csv('./data/initial/bike_rides.csv', index = False)

In [4]:
# query for commuters

query = """

with a as(

    select start_station_id
    , start_station_latitude
    , start_station_longitude
    , end_station_id
    , end_station_latitude
    , end_station_longitude
    , starttime
    , stoptime
    , tripduration
    , birth_year
    , gender
    FROM `bigquery-public-data.new_york_citibike.citibike_trips`
    where start_station_id is not null
    and end_station_id is not null
    and birth_year is not null
    and tripduration > 120
), b as (

    select start_station_id
    , end_station_id
    , starttime
    , stoptime
    , tripduration
    , birth_year
    , gender
    FROM `bigquery-public-data.new_york_citibike.citibike_trips`
    where start_station_id is not null
    and end_station_id is not null
    and birth_year is not null
    and tripduration > 120
)

select a.start_station_id as home
, a.start_station_latitude
, a.start_station_longitude
, a.end_station_id as work
, a.end_station_latitude
, a.end_station_longitude
, a.starttime as depart_home
, a.stoptime as arrive_work
, a.tripduration as commute_in
, b.starttime as depart_work
, b.stoptime as arrive_home
, b.tripduration as commute_out
from a join b on a.start_station_id = b.end_station_id
and a.end_station_id = b.start_station_id
and timestamp_trunc(timestamp(a.starttime), day, 'UTC') = timestamp_trunc(timestamp(b.starttime), day, 'UTC')
and a.starttime < b.starttime
and a.birth_year = b.birth_year
and a.gender = b.gender
and a.start_station_id <> b.start_station_id
and timestamp_diff(timestamp(b.starttime), timestamp(a.stoptime), hour) >= 6
"""

query_job = client.query(query)
results = query_job.result() 
df = results.to_dataframe()
df.to_csv('./data/initial/commuters.csv', index = False)